### Segmentation and Clustering

#### Imports

In [102]:
# Scrape tabular data from wikipedia
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


#### Scrape Wiki data

In [103]:
#Scrape wiki using beautiful soup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'}) #putting the data into a table based on the <table> tag

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4403a2be-3883-4510-be86-6a987a3c43c9","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

#### Create the dataframe

In [104]:
#Create Dataframe
df=pd.DataFrame(columns=['postal_code', 'borough','neighbourhood']) #initialize an empty dataframe
i=0 
rows = My_table.find_all('tr')

#extraxt data for each row
for row in rows:
    cells = row.find_all('td')
    if len(cells)>1:
        a=(cells[1].text.strip()!='Not assigned')#check if borough is 'Not assigned'
        b=(cells[2].text.strip()!='Not assigned')#check if neighbourhood is 'Not assigned'
        if (a):
            if(b):
                df.loc[i] = [cells[0].text.strip(),cells[1].text.strip(),cells[2].text.strip()]
            else: 
                df.loc[i] = [cells[0].text.strip(),cells[1].text.strip(),cells[1].text.strip()]
            i=i+1

#Merge Latitude and Longitude Data
csv='https://cocl.us/Geospatial_data'
geodata=pd.read_csv(csv)
geodata
df = pd.merge(df,geodata,left_on='postal_code',right_on='Postal Code', how='outer')
df.drop(['Postal Code'],axis=1,inplace=True)

df.head()

,postal_code,borough,neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Start the Analysis

#### import more packages

In [106]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

#### Use geopy library to get the latitude and longitude values of Toronto.

In [108]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_Ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [110]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, pc, nh in zip(df['Latitude'], df['Longitude'], df['postal_code'], df['neighbourhood']):
    label = '{}, {}'.format(pc, nh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Create a map of Toronto with bouroughs containing "Toronto" superimposed on top

In [124]:
df_Toronto = df[df['borough'].str.contains('Toronto')]
df_Toronto.reset_index(inplace=True)
df_Toronto.head()

,index,postal_code,borough,neighbourhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [125]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, pc, nh in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['postal_code'], df_Toronto['neighbourhood']):
    label = '{}, {}'.format(pc, nh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [126]:
CLIENT_ID = '3Q4RUVQHKBWVWCRIEP30U0Y41KNPCXTPZ2OP1BCMBVSCFEEG' # your Foursquare ID
CLIENT_SECRET = 'TTQRPNFXMBWRRQO2VQSQ4ALXO0ORPAJ54HSHAWRY55YJ3K2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3Q4RUVQHKBWVWCRIEP30U0Y41KNPCXTPZ2OP1BCMBVSCFEEG
CLIENT_SECRET:TTQRPNFXMBWRRQO2VQSQ4ALXO0ORPAJ54HSHAWRY55YJ3K2J


#### Exploring the first neighborhood in our dataframe.

In [183]:
neighbourhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_Toronto.loc[0, 'neighbourhood'] # neighborhood name

print('Latitude and longitude values of Neighbourhood {} are {}, {}.'.format(neighbourhood_name,
                                                                     neighbourhood_latitude,
                                                                     neighbourhood_longitude))

Latitude and longitude values of Neighbourhood Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Getting the top 30 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [184]:
# type your answer here
radius=500
LIMIT=20

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3Q4RUVQHKBWVWCRIEP30U0Y41KNPCXTPZ2OP1BCMBVSCFEEG&client_secret=TTQRPNFXMBWRRQO2VQSQ4ALXO0ORPAJ54HSHAWRY55YJ3K2J&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=20'

In [185]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb65939c6f59001bf528d0'},
 'response': {'venues': [{'id': '5bdc6c2bba57b4002c4c71a8',
    'name': 'Oldtown Bodega',
    'location': {'lat': 43.653966,
     'lng': -79.360752,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.653966,
       'lng': -79.360752}],
     'distance': 34,
     'postalCode': 'M5A 1L6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON M5A 1L6', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591436876',
    'hasPerk': False},
   {'id': '4bc70f5d14d7952126a066e9',
    'name': 'Sackville Playground',
    'location': {'address': '420 king st E',
     'lat': 43.65465604258614,
     'lng': -79.359870

In [186]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a *pandas* dataframe.

In [189]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.Categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.drop(['categories'], axis=1,inplace=True)
nearby_venues['categories']=nearby_venues['Categories']
nearby_venues.drop(['Categories'], axis=1,inplace=True)
nearby_venues.head()

,name,lat,lng,categories
0,Oldtown Bodega,43.653966,-79.360752,Café
1,Sackville Playground,43.654656,-79.359871,Park
2,Tandem Coffee,43.653559,-79.361809,Coffee Shop
3,TTC Streetcar #503 Kingston Rd,43.648099,-79.382932,Moving Target
4,Terroni Sud Forno Produzione e Spaccio,43.653903,-79.360018,Gourmet Shop


In [156]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


#### function to repeat the same process to all the neighborhoods in Toronto

In [224]:
def getNearbyVenues(neighbourhoods, latitudes, longitudes, radius=500):
    ven_list=pd.DataFrame(columns=['neighbourhood', 'latitude','longitude','name','lat','lng','categories'])
    c=0
    for neighbourhood, latitude,longitude in zip(neighbourhoods, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
        
        res = requests.get(url).json()['response']['venues']
        nearby_ven = json_normalize(res)
        # filter columns
        filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
        nearby_ven =nearby_ven.loc[:, filtered_columns]
        
        # filter the category for each row
        nearby_ven['venue.Categories'] = nearby_ven.apply(get_category_type, axis=1)
        
        # clean columns
        nearby_ven.columns = [col.split(".")[-1] for col in nearby_ven.columns]
        nearby_ven.drop(['categories'], axis=1,inplace=True)
        nearby_ven['categories']=nearby_ven['Categories']
        nearby_ven.drop(['Categories'], axis=1,inplace=True)
        nearby_ven['neighbourhood']=neighbourhood
        nearby_ven['latitude']=latitude
        nearby_ven['longitude']=longitude
        temp=nearby_ven[['neighbourhood', 'latitude','longitude','name','lat','lng','categories']]
        ven_list=ven_list.append(temp)
        #ven_list.append([(name,lat,lng,v['venues']['name'],v['venues']['location']['lat'],v['venues']['location']['lng'],v['venues']['categories'][0]['name']) for v in res])
    
    return (ven_list)


In [225]:
toronto_venues = getNearbyVenues(neighbourhoods=df_Toronto['neighbourhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )
toronto_venues.shape

(780, 7)

In [229]:
toronto_venues.head()

,neighbourhood,latitude,longitude,name,lat,lng,categories
0,"Regent Park, Harbourfront",43.65426,-79.360636,Oldtown Bodega,43.653966,-79.360752,Café
1,"Regent Park, Harbourfront",43.65426,-79.360636,Sackville Playground,43.654656,-79.359871,Park
2,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
3,"Regent Park, Harbourfront",43.65426,-79.360636,TTC Streetcar #503 Kingston Rd,43.648099,-79.382932,Moving Target
4,"Regent Park, Harbourfront",43.65426,-79.360636,Terroni Sud Forno Produzione e Spaccio,43.653903,-79.360018,Gourmet Shop


## 3. Analyze Each Neighborhood

In [232]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['neighbourhood'] = toronto_venues['neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,neighbourhood,ATM,Adult Boutique,Advertising Agency,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Alternative Healer,American Restaurant,Antique Shop,Art Gallery,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Beach,Bed & Breakfast,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Capitol Building,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Classroom,College Gym,College Lab,College Library,College Residence Hall,College Track,Comic Shop,Community Center,Concert Hall,Conference Room,Convenience Store,Costume Shop,Coworking Space,Deli / Bodega,Dentist's Office,Design Studio,Dessert Shop,Diner,Doctor's Office,Dog Run,Dry Cleaner,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Event Space,Fabric Shop,Factory,Falafel Restaurant,Fast Food Restaurant,Festival,Field,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General College & University,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,History Museum,Hospital,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Laundry Service,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Mosque,Motorcycle Shop,Moving Target,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Nursery School,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Parking,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Police Station,Pool,Post Office,Prayer Room,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Steakhouse,Storage Facility,Student Center,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,University,Video Game Store,Vietnamese Restaurant,Voting Booth,Warehouse,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [233]:
toronto_onehot.shape

(780, 234)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [234]:
toronto_grouped = toronto_onehot.groupby('neighbourhood').mean().reset_index()
toronto_grouped

,neighbourhood,ATM,Adult Boutique,Advertising Agency,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Alternative Healer,American Restaurant,Antique Shop,Art Gallery,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Beach,Bed & Breakfast,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Capitol Building,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Classroom,College Gym,College Lab,College Library,College Residence Hall,College Track,Comic Shop,Community Center,Concert Hall,Conference Room,Convenience Store,Costume Shop,Coworking Space,Deli / Bodega,Dentist's Office,Design Studio,Dessert Shop,Diner,Doctor's Office,Dog Run,Dry Cleaner,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Event Space,Fabric Shop,Factory,Falafel Restaurant,Fast Food Restaurant,Festival,Field,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General College & University,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,History Museum,Hospital,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Laundry Service,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Mosque,Motorcycle Shop,Moving Target,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Nursery School,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Parking,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Police Station,Pool,Post Office,Prayer Room,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Steakhouse,Storage Facility,Student Center,Supermarket,Surf Spot,Sushi Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,University,Video Game Store,Vietnamese Restaurant,Voting Booth,Warehouse,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,

In [235]:
toronto_grouped.shape

(39, 234)

#### Print each neighborhood along with the top 5 most common venues

In [236]:
num_top_venues = 5

for hood in toronto_grouped['neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
              venue  freq
0          Building  0.10
1            Office  0.10
2      Antique Shop  0.05
3  Sushi Restaurant  0.05
4        Food Truck  0.05


----Brockton, Parkdale Village, Exhibition Place----
                  venue  freq
0                Office  0.35
1          Tech Startup  0.15
2            Playground  0.05
3   Housing Development  0.05
4  Fast Food Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0   Government Building  0.10
1  Herbs & Spices Store  0.05
2         Auto Workshop  0.05
3           Pizza Place  0.05
4           Coffee Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0        Airport Gate  0.45
1                 Bar  0.05
2     Harbor / Marina  0.05
3             Airport  0.05
4  Airport Food Court  0.05


----Central Bay Street----


                                      venue  freq
0  Residential Building (Apartment / Condo)  0.15
1                       Rental Car Location  0.05
2                                     Hotel  0.05
3                                    Garden  0.05
4                           Assisted Living  0.05


----The Beaches----
                                      venue  freq
0                                      Park  0.10
1                                     Beach  0.05
2                                     Trail  0.05
3                                Playground  0.05
4  Residential Building (Apartment / Condo)  0.05


----The Danforth West, Riverdale----
               venue  freq
0     Ice Cream Shop  0.10
1  Health Food Store  0.10
2        Yoga Studio  0.05
3               Park  0.05
4         Nail Salon  0.05


----Toronto Dominion Centre, Design Exchange----
                venue  freq
0              Office  0.15
1                Park  0.15
2          Restaurant  0.10
3       Train 

In [237]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [244]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighbourhood'] = toronto_grouped['neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Office,Building,Bike Rental / Bike Share,Antique Shop,Library,Residential Building (Apartment / Condo),Bus Station,Student Center,Sushi Restaurant,Moving Target
1,"Brockton, Parkdale Village, Exhibition Place",Office,Tech Startup,Convenience Store,Spa,Fast Food Restaurant,Japanese Restaurant,Building,Residential Building (Apartment / Condo),Housing Development,Coffee Shop
2,"Business reply mail Processing Centre, South C...",Government Building,Theater,Light Rail Station,Building,Factory,Record Shop,Field,Fire Station,Storage Facility,Surf Spot
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Gate,Lake,Airport Lounge,Bar,Coffee Shop,Harbor / Marina,Airport Service,General Travel,Airport Food Court,Airport
4,Central Bay Street,Hospital,Coffee Shop,Auto Garage,Medical Center,Bike Rental / Bike Share,Metro Station,Dry Cleaner,Fast Food Restaurant,Burger Joint,Bus Line


## 4. Cluster Neighborhoods

In [245]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 2, 1, 0, 3, 0, 1, 3], dtype=int32)

In [246]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [247]:
toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('neighbourhood'), on='neighbourhood')

toronto_merged.head() # check the last columns!

,index,postal_code,borough,neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Moving Target,Furniture / Home Store,Library,Medical Center,Light Rail Station,Building,Mediterranean Restaurant,Spa,Italian Restaurant,Residential Building (Apartment / Condo)
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Government Building,Office,Medical Center,Sculpture Garden,Medical Lab,Capitol Building,Building,Thai Restaurant,General Entertainment,Monument / Landmark
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,School,Coffee Shop,Indian Restaurant,College Administrative Building,College Academic Building,Parking,Church,Pub,Mosque,Burrito Place
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Residential Building (Apartment / Condo),Building,Coffee Shop,Hostel,Park,Performing Arts Venue,Pharmacy,Nightclub,Salad Place,Diner
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Park,Coffee Shop,Beach,Playground,Residential Building (Apartment / Condo),Bakery,Trail,Diner,Dentist's Office,Design Studio


In [249]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [251]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,postal_code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5C,-79.375418,0,Residential Building (Apartment / Condo),Building,Coffee Shop,Hostel,Park,Performing Arts Venue,Pharmacy,Nightclub,Salad Place,Diner
5,M5E,-79.373306,0,Office,Building,Bike Rental / Bike Share,Antique Shop,Library,Residential Building (Apartment / Condo),Bus Station,Student Center,Sushi Restaurant,Moving Target
7,M6G,-79.422564,0,Grocery Store,Auto Dealership,Health & Beauty Service,Flower Shop,Laundry Service,Entertainment Service,Nightclub,College Classroom,Performing Arts Venue,Coffee Shop
8,M5H,-79.384568,0,Italian Restaurant,ATM,Concert Hall,Bar,Ballroom,Insurance Office,Café,BBQ Joint,Tea Room,Coworking Space
10,M5J,-79.381752,0,Residential Building (Apartment / Condo),Light Rail Station,Office,Indian Restaurant,Hotel,Coffee Shop,Pharmacy,Monument / Landmark,Middle Eastern Restaurant,Salad Place
16,M5L,-79.379817,0,Office,Food Court,Dentist's Office,Financial or Legal Service,Sushi Restaurant,Moving Target,Bank,History Museum,Building,Health Food Store
34,M5W,-79.374846,0,Residential Building (Apartment / Condo),Parking,Tech Startup,Office,Restaurant,Building,Bar,Italian Restaurant,Student Center,Hotel
36,M5X,-79.382280,0,Bakery,Grocery Store,Clothing Store,Bubble Tea Shop,Salad Place,Soup Place,Doctor's Office,Juice Bar,Flower Shop,Food Court


#### Cluster 2

In [252]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,postal_code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,-79.378937,1,School,Coffee Shop,Indian Restaurant,College Administrative Building,College Academic Building,Parking,Church,Pub,Mosque,Burrito Place
6,M5G,-79.387383,1,Hospital,Coffee Shop,Auto Garage,Medical Center,Bike Rental / Bike Share,Metro Station,Dry Cleaner,Fast Food Restaurant,Burger Joint,Bus Line
11,M6J,-79.419750,1,Coffee Shop,Mac & Cheese Joint,Japanese Restaurant,Butcher,Festival,Bar,Brewery,Beer Store,Thai Restaurant,Club House
17,M4M,-79.340923,1,Coffee Shop,School,Smoothie Shop,Ice Cream Shop,BBQ Joint,Butcher,Restaurant,Candy Store,Cheese Shop,Real Estate Office
23,M4R,-79.405678,1,Coffee Shop,Shoe Store,Sporting Goods Shop,General Entertainment,Ice Cream Shop,Fast Food Restaurant,Metro Station,Conference Room,Playground,Church
26,M4S,-79.388790,1,Coffee Shop,Italian Restaurant,Candy Store,Gift Shop,Nail Salon,Library,Spa,Dessert Shop,Japanese Restaurant,Costume Shop
27,M5S,-79.400049,1,College Gym,Coffee Shop,College Classroom,Student Center,Food Truck,College Lab,College Library,College Residence Hall,College Track,Medical School
28,M6S,-79.484450,1,Café,Coffee Shop,Salon / Barbershop,Falafel Restaurant,Bakery,Optical Shop,Doctor's Office,Diner,School,Medical Center
30,M5T,-79.400049,1,Thrift / Vintage Store,Organic Grocery,Hostel,Boutique,Shoe Store,Liquor Store,Salon / Barbershop,Bar,Speakeasy,Bookstore
35,M4X,-79.367675,1,Coffee Shop,Bakery,Ramen Restaurant,Laundry Service,Design Studio,Miscellaneous Shop,Ice Cream Shop,Market,Flower Shop,Spa


#### Cluster 3

In [253]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,postal_code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5V,-79.39442,2,Airport Gate,Lake,Airport Lounge,Bar,Coffee Shop,Harbor / Marina,Airport Service,General Travel,Airport Food Court,Airport


#### Cluster 4

In [254]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,postal_code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,3,Moving Target,Furniture / Home Store,Library,Medical Center,Light Rail Station,Building,Mediterranean Restaurant,Spa,Italian Restaurant,Residential Building (Apartment / Condo)
1,M7A,-79.389494,3,Government Building,Office,Medical Center,Sculpture Garden,Medical Lab,Capitol Building,Building,Thai Restaurant,General Entertainment,Monument / Landmark
4,M4E,-79.293031,3,Park,Coffee Shop,Beach,Playground,Residential Building (Apartment / Condo),Bakery,Trail,Diner,Dentist's Office,Design Studio
9,M6H,-79.442259,3,Park,Shopping Mall,Music Venue,Fabric Shop,Speakeasy,Bank,Fast Food Restaurant,Automotive Shop,Community Center,Post Office
12,M4K,-79.352188,3,Health Food Store,Ice Cream Shop,Yoga Studio,Tanning Salon,Greek Restaurant,Fruit & Vegetable Store,Elementary School,Metro Station,Nail Salon,Park
13,M5K,-79.381576,3,Park,Office,Restaurant,Deli / Bodega,Food Court,Parking,Italian Restaurant,Cocktail Bar,Nursery School,Hotel
15,M4L,-79.315572,3,Park,Coffee Shop,Fish & Chips Shop,Pet Store,Playground,Police Station,Church,Racetrack,Miscellaneous Shop,Bus Line
18,M4N,-79.388790,3,Bus Line,Salon / Barbershop,Bus Stop,Fast Food Restaurant,Hospital,Prayer Room,High School,Gym / Fitness Center,Snack Place,Sandwich Place
19,M5N,-79.416936,3,Doctor's Office,Gym,Italian Restaurant,Playground,High School,Office,Yoga Studio,Design Studio,Dessert Shop,Diner
20,M4P,-79.390197,3,Residential Building (Apartment / Condo),Dog Run,Convenience Store,Dentist's Office,Doctor's Office,Office,Park,Voting Booth,Electronics Store,Breakfast Spot


#### Cluster 5

In [255]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,postal_code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M6K,-79.428191,4,Office,Tech Startup,Convenience Store,Spa,Fast Food Restaurant,Japanese Restaurant,Building,Residential Building (Apartment / Condo),Housing Development,Coffee Shop
